<img src="./images/DLI_Header.png" style="width: 400px;">

# Assessment

Congratulations on going through today's course! Hopefully, you've learned some valuable skills along the way and had fun doing it. Now it's time to put those skills to the test. In this assessment, you will train a new model that is able to recognize fresh and rotten fruit. You will need to get the model to a validation accuracy of `92%` in order to pass the assessment, though we challenge you to do even better if you can. You will have the use the skills that you learned in the previous exercises. Specifically, we suggest using some combination of transfer learning, data augmentation, and fine tuning. Once you have trained the model to be at least 92% accurate on the validation dataset, save your model, and then assess its accuracy. Let's get started! 

## The Dataset

In this exercise, you will train a model to recognize fresh and rotten fruits. The dataset comes from [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification), a great place to go if you're interested in starting a project after this class. The dataset structure is in the `data/fruits` folder. There are 6 categories of fruits: fresh apples, fresh oranges, fresh bananas, rotten apples, rotten oranges, and rotten bananas. This will mean that your model will require an output layer of 6 neurons to do the categorization successfully. You'll also need to compile the model with `categorical_crossentropy`, as we have more than two categories.

<img src="./images/fruits.png" style="width: 600px;">

## Load ImageNet Base Model

We encourage you to start with a model pretrained on ImageNet. Load the model with the correct weights, set an input shape, and choose to remove the last layers of the model. Remember that images have three dimensions: a height, and width, and a number of channels. Because these pictures are in color, there will be three channels for red, green, and blue. We've filled in the input shape for you. This cannot be changed or the assessment will fail. If you need a reference for setting up the pretrained model, please take a look at [notebook 05b](05b_presidential_doggy_door.ipynb) where we implemented transfer learning.

In [1]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

2024-01-26 01:36:23.092815: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-26 01:36:23.188070: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-26 01:36:23.188091: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-26 01:36:23.213438: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-26 01:36:23.249390: I tensorflow/core/platform/cpu_feature_guar

## Freeze Base Model

Next, we suggest freezing the base model, as done in [notebook 05b](05b_presidential_doggy_door.ipynb). This is done so that all the learning from the ImageNet dataset does not get destroyed in the initial training.

In [2]:
# Freeze base model
base_model.trainable = False

## Add Layers to Model

Now it's time to add layers to the pretrained model. [Notebook 05b](05b_presidential_doggy_door.ipynb) can be used as a guide. Pay close attention to the last dense layer and make sure it has the correct number of neurons to classify the different types of fruit.

In [3]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 6)                 3078      
                                                                 
Total params: 14717766 (56.14 MB)
Trainable params: 3078 (12.02 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


## Compile Model

Now it's time to compile the model with loss and metrics options. Remember that we're training on a number of different categories, rather than a binary classification problem.

In [5]:
model.compile(loss='categorical_crossentropy' , metrics=['accuracy'])

## Augment the Data

If you'd like, try to augment the data to improve the dataset. Feel free to look at [notebook 04a](04a_asl_augmentation.ipynb) and [notebook 05b](05b_presidential_doggy_door.ipynb) for augmentation examples. There is also documentation for the [Keras ImageDataGenerator class](https://keras.io/api/preprocessing/image/#imagedatagenerator-class). This step is optional, but it may be helpful to get to 92% accuracy.

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(samplewise_center=True,
                            rotation_range=10,
                            zoom_range = 0.1,
                            width_shift_range=0.1, 
                            height_shift_range=0.1,  
                            horizontal_flip=True, 
                            vertical_flip=False)

## Load Dataset

Now it's time to load the train and validation datasets. Pick the right folders, as well as the right `target_size` of the images (it needs to match the height and width input of the model you've created). If you'd like a reference, you can check out [notebook 05b](05b_presidential_doggy_door.ipynb).

In [7]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('data/fruits/train/', 
                                       target_size=(224,224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('data/fruits/valid/', 
                                      target_size=(224,224), 
                                      color_mode='rgb', 
                                      class_mode="categorical")

Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


## Train the Model

Time to train the model! Pass the `train` and `valid` iterators into the `fit` function, as well as setting your desired number of epochs.

In [9]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=30)

Epoch 1/30


2024-01-26 01:45:12.335928: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8800
2024-01-26 01:45:14.470877: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f0d752c9800 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-26 01:45:14.470912: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-01-26 01:45:14.503730: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


 3/36 [=>............................] - ETA: 2s - loss: 5.9298 - accuracy: 0.1771

I0000 00:00:1706251514.678402 1812512 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


36/36 [==============================] - 21s 498ms/step - loss: 2.3561 - accuracy: 0.5000 - val_loss: 1.1692 - val_accuracy: 0.6778
Epoch 2/30
36/36 [==============================] - 16s 436ms/step - loss: 0.6895 - accuracy: 0.7733 - val_loss: 0.6003 - val_accuracy: 0.8176
Epoch 3/30
36/36 [==============================] - 17s 456ms/step - loss: 0.3424 - accuracy: 0.8849 - val_loss: 0.4055 - val_accuracy: 0.8784
Epoch 4/30
36/36 [==============================] - 17s 450ms/step - loss: 0.2307 - accuracy: 0.9171 - val_loss: 0.3091 - val_accuracy: 0.9210
Epoch 5/30
36/36 [==============================] - 17s 461ms/step - loss: 0.1618 - accuracy: 0.9416 - val_loss: 0.2496 - val_accuracy: 0.9149
Epoch 6/30
36/36 [==============================] - 31s 847ms/step - loss: 0.1192 - accuracy: 0.9594 - val_loss: 0.3562 - val_accuracy: 0.9027
Epoch 7/30
36/36 [==============================] - 17s 463ms/step - loss: 0.1085 - accuracy: 0.9611 - val_loss: 0.1742 - val_accuracy: 0.9514
Epoch 8/30

## Unfreeze Model for Fine Tuning

If you have reached 92% validation accuracy already, this next step is optional. If not, we suggest fine tuning the model with a very low learning rate.

In [10]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = 0.00001),
              loss ='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=16)

Epoch 1/16
36/36 [==============================] - 41s 713ms/step - loss: 0.0395 - accuracy: 0.9898 - val_loss: 0.1241 - val_accuracy: 0.9696
Epoch 2/16
36/36 [==============================] - 16s 437ms/step - loss: 0.0342 - accuracy: 0.9898 - val_loss: 0.1452 - val_accuracy: 0.9635
Epoch 3/16
36/36 [==============================] - 17s 449ms/step - loss: 0.0093 - accuracy: 0.9966 - val_loss: 0.0888 - val_accuracy: 0.9848
Epoch 4/16
36/36 [==============================] - 16s 433ms/step - loss: 0.0182 - accuracy: 0.9966 - val_loss: 0.1721 - val_accuracy: 0.9666
Epoch 5/16
36/36 [==============================] - 16s 433ms/step - loss: 0.0020 - accuracy: 0.9992 - val_loss: 0.3525 - val_accuracy: 0.9514
Epoch 6/16
36/36 [==============================] - 16s 430ms/step - loss: 0.0126 - accuracy: 0.9966 - val_loss: 0.1815 - val_accuracy: 0.9726
Epoch 7/16
36/36 [==============================] - 16s 427ms/step - loss: 5.6077e-04 - accuracy: 1.0000 - val_loss: 0.1431 - val_accuracy: 0.

## Evaluate the Model

Hopefully, you now have a model that has a validation accuracy of 92% or higher. If not, you may want to go back and either run more epochs of training, or adjust your data augmentation. 

Once you are satisfied with the validation accuracy, evaluate the model by executing the following cell. The evaluate function will return a tuple, where the first value is your loss, and the second value is your accuracy. To pass, the model will need have an accuracy value of `92% or higher`. 

In [12]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

10/10 [==============================] - 4s 355ms/step - loss: 0.1267 - accuracy: 0.9818


[0.12666772305965424, 0.9817629456520081]

## Run the Assessment

To assess your model run the following two cells.

**NOTE:** `run_assessment` assumes your model is named `model` and your validation data iterator is called `valid_it`. If for any reason you have modified these variable names, please update the names of the arguments passed to `run_assessment`.

In [ ]:
from run_assessment import run_assessment

In [ ]:
run_assessment(model, valid_it)

## Generate a Certificate

If you passed the assessment, please return to the course page (shown below) and click the "ASSESS TASK" button, which will generate your certificate for the course.

<img src="./images/assess_task.png" style="width: 800px;">